# Restart an idr-analysis JupyterHub server on Kubernetes

Occasionally the JupyterHub server will fail to spawn new pods for unknown reasons.
When this happens uses can connect to JupyterHub, but starting a server fails with a 500 error.
The error message may sometimes be misleading.

When this happens try the following.

## Clear state in a user's web browser

Clear any cookies or sessions associated with idr-analysis in the user's browser.
For example, in Firefox you can `Forget About This Site` in the history.
Attempt to connect to JupyterHub again.

## Investigate JupyterHub

If clearing the user's browser setting does not resolve the issue you will need to investigate JupyterHub.


### Kubernetes configuration

First ensure you can connect to Kubernetes.
If you are using this Docker image and followed the instructions to run it `kubectl` is already installed and configured for connection to the Kubernetes cluster.

If you need to install and configure it yourself
- Go to https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl and follow the instructions to install kubectl `1.9.*` or `1.10.*`
- Decrypt the Kubernetes configuration file: `pass show secret/k8s/config > kube.config`
- Set the `KUBECONFIG` environment variable to point to this file: `export KUBECONFIG=kube.config`

Check you can connect to the Kubernetes cluster:

In [2]:
kubectl get nodes

NAME                 STATUS    ROLES     AGE       VERSION
k8s01-vae-master-1   Ready     master    131d      v1.9.3+coreos.0
k8s01-vae-master-2   Ready     master    131d      v1.9.3+coreos.0
k8s01-vae-master-3   Ready     master    131d      v1.9.3+coreos.0
k8s01-vae-node-1     Ready     node      131d      v1.9.3+coreos.0
k8s01-vae-node-2     Ready     node      131d      v1.9.3+coreos.0
k8s01-vae-node-3     Ready     node      131d      v1.9.3+coreos.0


### Check JupyterHub logs
Change the `--since argument` to a longer time frame, e.g. `1d`

In [5]:
kubectl -n jupyterhub-itr-public logs deploy/hub --since 5m

[I 2018-08-09 15:07:26.362 JupyterHub proxy:301] Checking routes
[I 2018-08-09 15:08:26.360 JupyterHub proxy:301] Checking routes
[I 2018-08-09 15:09:26.360 JupyterHub proxy:301] Checking routes
[I 2018-08-09 15:10:26.361 JupyterHub proxy:301] Checking routes
[I 2018-08-09 15:10:26.525 JupyterHub log:158] 200 GET /itr-public/hub/api/users (cull-idle@127.0.0.1) 6.35ms
[I 2018-08-09 15:11:26.362 JupyterHub proxy:301] Checking routes


If the reason for the outage is unclear from the logs restart the hub

In [6]:
kubectl -n jupyterhub-itr-public get pods --selector=component=hub

NAME                  READY     STATUS    RESTARTS   AGE
hub-75bd7db47-gvt59   1/1       Running   0          22h


In [9]:
kubectl -n jupyterhub-itr-public delete pods --selector=component=hub

pod "hub-75bd7db47-hbwnq" deleted


You should also delete any single user servers that are still running as the JupyterHub database is shelp in memory so will be lost by the restart

In [10]:
kubectl -n jupyterhub-itr-public get pods --selector=component=singleuser-server

NAME                                                   READY     STATUS    RESTARTS   AGE
jupyter-66179dc6-2dd929-2d48c6-2db8dc-2df4bc26dc70a5   1/1       Running   0          22h
jupyter-6a903d58-2dd595-2d48ca-2dbdf2-2d17f79fbf8ba0   1/1       Running   0          5h


In [11]:
kubectl -n jupyterhub-itr-public delete pods --selector=component=singleuser-server

pod "jupyter-66179dc6-2dd929-2d48c6-2db8dc-2df4bc26dc70a5" deleted
pod "jupyter-6a903d58-2dd595-2d48ca-2dbdf2-2d17f79fbf8ba0" deleted


Hopefully everything is working now!

In [12]:
kubectl -n jupyterhub-itr-public get pods --selector=component=hub

NAME                  READY     STATUS    RESTARTS   AGE
hub-75bd7db47-pjpcc   1/1       Running   0          1m
